In [467]:
import pandas as pd
import numpy as np
dimension_path = "../DimensionsPre/"
data_path = "../Data/"

In [468]:
Date_dimension = pd.read_csv(dimension_path+"Date_dimension/Date_dimension.csv")
Mobility_dimension = pd.read_csv(dimension_path+"Mobility_dimension/Mobility_dimension.csv")
Patient_dimension = pd.read_csv(dimension_path+"Patient_dimension/Patient_dimension.csv")
PHU_Location_dimension = pd.read_csv(dimension_path+"PHU_Location_dimension/PHU_Location_dimension.csv")
Special_Measures_dimension = pd.read_csv(dimension_path+"Special_Measures_dimension/Special_Measures_dimension.csv")
Weather_dimension = pd.read_csv(dimension_path+"Weather_dimension/Weather_dimension.csv")

In [469]:
cases = pd.read_csv(data_path+"IndividualCasesInOntario.csv")
cases.head()

,_id,Row_ID,Accurate_Episode_Date,Case_Reported_Date,Test_Reported_Date,Specimen_Date,Age_Group,Client_Gender,Case_AcquisitionInfo,Outcome1,Outbreak_Related,Reporting_PHU_ID,Reporting_PHU,Reporting_PHU_Address,Reporting_PHU_City,Reporting_PHU_Postal_Code,Reporting_PHU_Website,Reporting_PHU_Latitude,Reporting_PHU_Longitude
0,1,1,2020-06-04T00:00:00,2020-06-06T00:00:00,2020-06-06T00:00:00,2020-06-04T00:00:00,50s,FEMALE,CC,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
1,2,2,2020-06-01T00:00:00,2020-06-03T00:00:00,2020-06-03T00:00:00,2020-06-02T00:00:00,30s,MALE,CC,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
2,3,3,2020-05-30T00:00:00,2020-06-02T00:00:00,2020-06-02T00:00:00,2020-05-30T00:00:00,20s,MALE,CC,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
3,4,4,2020-04-15T00:00:00,2020-04-17T00:00:00,2020-04-17T00:00:00,2020-04-16T00:00:00,80s,FEMALE,OB,Resolved,Yes,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
4,5,5,2020-04-18T00:00:00,2020-04-21T00:00:00,2020-04-21T00:00:00,2020-04-18T00:00:00,20s,MALE,OB,Resolved,Yes,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893


In [470]:
cases = cases[(cases["Reporting_PHU_City"]=="Toronto")|(cases["Reporting_PHU_City"]=="Ottawa")]


In [471]:
cases["Accurate_Episode_Date"] = pd.to_datetime(cases["Accurate_Episode_Date"]).dt.strftime('%Y-%m-%d')
cases["Case_Reported_Date"] = pd.to_datetime(cases["Case_Reported_Date"]).dt.strftime('%Y-%m-%d')
cases["Test_Reported_Date"] = pd.to_datetime(cases["Test_Reported_Date"]).dt.strftime('%Y-%m-%d')
cases["Specimen_Date"] = pd.to_datetime(cases["Specimen_Date"]).dt.strftime('%Y-%m-%d')

cases = cases.fillna({'Outbreak_Related':'Unknown'})[:10000]

In [472]:
cases["Reporting_PHU_City"].isna().sum()

0

In [473]:
# fact_table =  pd.DataFrame(columns=['Onset_date_key','Reported_date_key','Test_date_key', 'Specimen_date_key', 'Patient_key',"PHU_location_key","Mobility_key","Special_measure_key","Weather_key","Resolved","Unresolved","Fatal"])
# fact_table =  pd.DataFrame(columns=['Onset_date_key','Reported_date_key','Test_date_key', 'Specimen_date_key', 'Patient_key'])
fact_table =  pd.DataFrame(columns=['Onset_date_key','Reported_date_key','Test_date_key', 'Specimen_date_key', 'Patient_key', "PHU_location_key","Special_measure_key","Mobility_key","Weather_key","Resolved","Unresolved","Fatal"])
# fact_table =  pd.DataFrame(columns=['Onset_date_key']


In [474]:
fact_table.head()

,Onset_date_key,Reported_date_key,Test_date_key,Specimen_date_key,Patient_key,PHU_location_key,Special_measure_key,Mobility_key,Weather_key,Resolved,Unresolved,Fatal


In [ ]:
#TO REMOVED:df.dropna(axis=0,how='any')
cases = cases.dropna(axis=0,how='any')
# cases = cases.head()
rows = 0
Resolved,Unresolved,Fatal = 0,0,0




for idx, row in cases.iterrows():
    rows += 1
    
    #not considering 2021 data
    if (row["Case_Reported_Date"][:4]!="2020"):
        continue
    
    
    percentage = str(round((rows/len(cases))*100, 2)  )
    print("%s/%s - %s%% "%(str(rows),str(len(cases)),percentage),end = "\r")
    
    if (row["Outcome1"] == "Resolved"):
        Resolved+=1
    elif (row["Outcome1"] == "Not Resolved"):
        Unresolved+=1
    else:
        Fatal+=1

    fact_row = [
        Date_dimension[Date_dimension["date"]==row["Accurate_Episode_Date"]]["surrogate_key"].values[0],
        Date_dimension[Date_dimension["date"]==row["Case_Reported_Date"]]["surrogate_key"].values[0],
        Date_dimension[Date_dimension["date"]==row["Test_Reported_Date"]]["surrogate_key"].values[0],
        Date_dimension[Date_dimension["date"]==row["Specimen_Date"]]["surrogate_key"].values[0],
        Patient_dimension[(Patient_dimension["Age_Group"]==row["Age_Group"])&(Patient_dimension["Client_Gender"]==row["Client_Gender"])&(Patient_dimension["Case_AcquisitionInfo"]==row["Case_AcquisitionInfo"])&(Patient_dimension["Outbreak_Related"]==row["Outbreak_Related"])]["surrogate_key"].values[0],
        PHU_Location_dimension[PHU_Location_dimension["Reporting_PHU_ID"]==row["Reporting_PHU_ID"]]["surrogate_key"].values[0],
        "TODO special",
        Mobility_dimension[(Mobility_dimension["sub_region_2"]==row["Reporting_PHU_City"])&(Mobility_dimension["date"]==row["Case_Reported_Date"])]["surrogate_key"].values[0], #TODO Case_Reported_Date???
        Weather_dimension[(Weather_dimension["Station Name"]==row["Reporting_PHU_City"])&(Weather_dimension["Date/Time"]==row["Case_Reported_Date"])]["surrogate_key"].values[0], 
        Resolved,Unresolved,Fatal

               
        ]
    fact_table.loc[len(fact_table)] = fact_row
    
        
        
        
        
        
        

In [ ]:
cases["Test_Reported_Date"].drop_duplicates()

In [ ]:
len(cases)

In [ ]:
fact_table.shape

In [ ]:
fact_table

In [ ]:
cases["Outcome1"].drop_duplicates()

In [ ]:
fact_table.to_csv("Covid-19_Tracking_Fact_Table.csv",index=False)

In [ ]:
Mobility_dimension["sub_region_2"].drop_duplicates()

In [ ]:
cases["Reporting_PHU_City"].drop_duplicates()

In [ ]:
Weather_dimension["Station Name"].drop_duplicates()

In [ ]:
Weather_dimension["Station Name"].isna().sum()

In [ ]:
cases["Reporting_PHU_City"].isna().sum()

In [ ]:
cases["Reporting_PHU_City"].drop_duplicates()

In [ ]:
ottawa

In [ ]:
cases[cases["Reporting_PHU_City"]=="Toronto"].sum()

In [ ]:
# Weather_dimension["Station Name"]==
cases["Reporting_PHU_City"].drop_duplicates()


In [ ]:
Weather_dimension["Date/Time"].isna().sum()
cases["Case_Reported_Date"].isna().sum()